# Example-05: Sextupole element factory

In [1]:
# In this example sextupole factory is illustrated

In [2]:
import jax
from jax import jit
from jax import jacrev

from elementary.util import ptc
from elementary.util import beta
from elementary.sextupole import sextupole_factory

jax.numpy.set_printoptions(linewidth=256, precision=12)

In [3]:
# Set data type

jax.config.update("jax_enable_x64", True)

In [4]:
# Set device

device, *_ = jax.devices('cpu')
jax.config.update('jax_default_device', device)

In [5]:
# Set initial condition

(q_x, q_y, q_s) = qs = jax.numpy.array([-0.01, 0.005, 0.001])
(p_x, p_y, p_s) = ps = jax.numpy.array([0.001, 0.001, -0.0001])
x = jax.numpy.hstack([qs, ps])

In [6]:
# Define generic sextupole element

gamma = 10**6
element = jit(sextupole_factory(beta=beta(gamma), gamma=gamma, order=2**1, iterations=100, settings=dict(ns=2**1)))

In [7]:
# Compare with PTC

length = jax.numpy.float64(0.2)
kn = jax.numpy.float64(-50.0)
ks = jax.numpy.float64(+75.0)

print(res := element(x, length, kn, ks))
print(ref := ptc(x, 'sextupole', {'l': float(length), 'k2': float(kn), 'k2s': float(ks)}, gamma=gamma))
print(jax.numpy.allclose(res, ref))

[-0.009839311476  0.005305274154  0.000999691958  0.000595944971  0.002048184556 -0.0001        ]
[-0.009839311476  0.005305274154  0.000999691958  0.000595944971  0.002048184556 -0.0001        ]
True


In [8]:
# Differentiability

print(jacrev(element)(x, length, kn, ks))
print()

print(jacrev(element, 1)(x, length, kn, ks))
print()

[[ 9.977539230852e-01 -1.999683875914e-02  0.000000000000e+00  1.998755038415e-01 -1.333336463861e-03 -1.588878018948e-04]
 [-1.999528286317e-02  1.002381087687e+00  0.000000000000e+00 -1.333283393123e-03  2.001711272351e-01 -3.042892476466e-04]
 [ 3.567742993840e-05  1.067886227030e-05  1.000000000000e+00 -1.582181525155e-04 -3.046374640722e-04  6.130677412435e-07]
 [-2.075140977984e-02 -2.000692596676e-01  0.000000000000e+00  9.979602547484e-01 -2.001825891300e-02  2.884095678113e-05]
 [-2.000275114867e-01  2.345269216812e-02  0.000000000000e+00 -2.001650181601e-02  1.002174743629e+00  1.451546593510e-05]
 [-1.222214716319e-23  6.594282317221e-24  0.000000000000e+00  7.463717130175e-24 -6.770206730037e-24  1.000000000000e+00]]

[ 5.960059281979e-04  2.048394055709e-03 -2.275568047834e-06 -2.198365452739e-03  5.184991611242e-03  1.898300850487e-25]

